# Blind Z Gate

In the following we exemplify how to implement an X gate with `pytket-mbqc`.

We first initialise a graph state with 2 physical qubits and 3 logical qubits. The number of logical qubits can be higher than the number of phisical qubits if qubits are reused, as is the case here.

In [ ]:
from pytket_mbqc_py import GraphCircuit
from pytket.circuit.display import render_circuit_jupyter

graph_circuit = GraphCircuit(
    n_physical_qubits=2,
    n_logical_qubits=3,
)
render_circuit_jupyter(graph_circuit)

Note that before any operations constructing a graph state have been added there is a round of randomness generation. This randomness is saved in the `rand` registers.

Next we create a linear cluster state consisting of 3 qubits. Note that vertices are measured once all of the edges they connect to are added. This allows for some qubit reuse.

The measurement angles differ between vertices, being equivelent to an Z gate. The input is |+>, and the measurements are in the hadamard bases.

In [ ]:
import networkx as nx

input_vertex = graph_circuit.add_graph_vertex(measurement_order=0)

graph_vertex = graph_circuit.add_graph_vertex(measurement_order=None)
graph_circuit.add_edge(input_vertex, graph_vertex)
graph_circuit.corrected_measure(vertex=input_vertex, t_multiple=4)

output_vertex = graph_circuit.add_graph_vertex(measurement_order=None)
graph_circuit.add_edge(graph_vertex, output_vertex)
graph_circuit.corrected_measure(vertex=graph_vertex, t_multiple=4)

graph_circuit.corrected_measure(vertex=output_vertex, t_multiple=0)

nx.draw(graph_circuit.entanglement_graph, with_labels=True)

In [ ]:
render_circuit_jupyter(graph_circuit)

We can now run the resulting circuit, giving the 1 state 100% of the time, as we expect.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline

api_offline = QuantinuumAPIOffline()
backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)

output_reg = [graph_circuit.vertex_reg[output_vertex][0]]

compiled_circuit = backend.get_compiled_circuit(graph_circuit)
n_shots = 100
result = backend.run_circuit(circuit=compiled_circuit, n_shots=n_shots)
result.get_counts(cbits=output_reg)